In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from torch.utils.data import DataLoader, TensorDataset

In [2]:
iris = load_iris()
X = iris.data
y = iris.target

X = X[y != 2]
y = y[y != 2]

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [4]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [5]:
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

In [6]:
class RegularizedNN(nn.Module):
    def __init__(self):
        super(RegularizedNN, self).__init__()
        self.fc1 = nn.Linear(4, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 2)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return torch.softmax(x, dim=-1)

In [7]:
best_val_loss = float('inf')
early_stop_count = 0
patience = 10

model = RegularizedNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.01)

In [8]:

def add_noise(X, noise_factor=0.1):
    return X + noise_factor * torch.randn_like(X)

num_epochs = 100
for epoch in range(num_epochs):
    model.train()

    X_train_augmented = add_noise(X_train_tensor)

    outputs = model(X_train_augmented)
    loss = criterion(outputs, y_train_tensor)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    model.eval()
    with torch.no_grad():
        val_outputs = model(X_test_tensor)
        val_loss = criterion(val_outputs, y_test_tensor)

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        early_stop_count = 0
    else:
        early_stop_count += 1
        if early_stop_count >= patience:
            print(f"Early stopping at epoch {epoch+1}")
            break

    if epoch % 10 == 0:
        print(f"Epoch [{epoch}/{num_epochs}], Loss: {loss.item():.4f}, Validation Loss: {val_loss.item():.4f}")

model.eval()
with torch.no_grad():
    y_pred_prob = model(X_test_tensor)
    y_pred = torch.argmax(y_pred_prob, dim=1)

Epoch [0/100], Loss: 0.7550, Validation Loss: 0.7643
Epoch [10/100], Loss: 0.6551, Validation Loss: 0.6587
Epoch [20/100], Loss: 0.5828, Validation Loss: 0.5604
Epoch [30/100], Loss: 0.5044, Validation Loss: 0.4745
Epoch [40/100], Loss: 0.4312, Validation Loss: 0.4070
Epoch [50/100], Loss: 0.3827, Validation Loss: 0.3647
Epoch [60/100], Loss: 0.3600, Validation Loss: 0.3423
Epoch [70/100], Loss: 0.3571, Validation Loss: 0.3315
Epoch [80/100], Loss: 0.3371, Validation Loss: 0.3262
Epoch [90/100], Loss: 0.3316, Validation Loss: 0.3236


In [9]:
accuracy = accuracy_score(y_test_tensor.numpy(), y_pred.numpy())
print(f"\nTest Accuracy: {accuracy:.4f}")


Test Accuracy: 1.0000
